In [3]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial

#read metadata, and add columns for additional information
csvPath='/home/sliceruser/labels/clinical_information/marksheet.csv'
df = pd.read_csv(csvPath)
#initializing empty columns
df["reSampledPath"] = ""
df["adc"] = ""
df["cor"] = ""
df["hbv"] = ""
df["sag"] = ""
df["t2w"] = ""
df["isAnythingInAnnotated"] = 0
df["isAnyMissing"] = False

targetDir= '/home/sliceruser/data/orig'

dirDict={}
for subdir, dirs, files in os.walk(targetDir):
    for subdirin, dirsin, filesin in os.walk(subdir):
        lenn= len(filesin)
        if(lenn>0):
            dirDict[subdirin.split("/")[5]]=filesin

labelsFiles=[]
labelsRootDir = '/home/sliceruser/labels/csPCa_lesion_delineations/human_expert/resampled/'
for subdir, dirs, files in os.walk(labelsRootDir):
    labelsFiles=files
    
#Constructing functions that when applied to each row will fill the necessary path data
listOfDeficientStudyIds=[]



df

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,reSampledPath,adc,cor,hbv,sag,t2w,isAnythingInAnnotated,isAnyMissing
0,10000,1000000,2019-07-02,73,7.70,NaN,55.0,MRBx,0+0,,,,,,,0,False
1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,,,,,,,0,False
2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,,,,,,,0,False
3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,,,,,,,0,False
4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0",,,,,,,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3",,,,,,,0,False
1496,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,,,,,,,0,False
1497,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,,,,,,,0,False
1498,11474,1001498,2016-05-03,71,12.00,NaN,83.0,MRBx,3+3,,,,,,,0,False


In [6]:
def findPathh(row,dirDictt,keyWord,targetDir):
    row=row[1]
    patId=str(row['patient_id'])
    study_id=str(row['study_id'])
    #first check is such key present
    if(patId in dirDictt ):
        filtered = list(filter(lambda file_name:   (keyWord in file_name and  study_id  in  file_name  ), dirDictt[patId]  ))
        if(len(filtered)>0):
            return os.path.join(targetDir,  patId, filtered[0] )
        else:
            print(f"no {keyWord} in {study_id}")
            listOfDeficientStudyIds.append(study_id)
            return " " 
    listOfDeficientStudyIds.append(study_id)
    return " "

def iter_paths_apply(dff,keyword):
    resList=[]
    with mp.Pool(processes = mp.cpu_count()) as pool:
        resList=pool.map(partial(findPathh,dirDictt=dirDict,keyWord='t2w',targetDir=targetDir)  ,list(dff.iterrows()))
    dff[keyword]=resList   

iter_paths_apply(df,'t2w')
iter_paths_apply(df,'adc')
iter_paths_apply(df,'hbv')
iter_paths_apply(df,'sag')


def findResampledLabel(row,labelsFiles):
    patId=str(row['patient_id'])
    study_id=str(row['study_id'])
    filtered = list(filter(lambda file_name:   (study_id  in  file_name  ), labelsFiles ))
    if(len(filtered)>0):
        return os.path.join(labelsRootDir, filtered[0])
    listOfDeficientStudyIds.append(study_id)    
    return " "
    
        
df["reSampledPath"] =  df.apply(lambda row : findResampledLabel(row,labelsFiles )   , axis = 1)  

def isAnythingInAnnotated(row):
    row=row[1]
    reSampledPath=str(row['reSampledPath'])
    if(len(reSampledPath)>1):
        image = sitk.ReadImage(reSampledPath)
        nda = sitk.GetArrayFromImage(image)
        return np.sum(nda)
    return 0

resList=[]
with mp.Pool(processes = mp.cpu_count()) as pool:
    resList=pool.map(isAnythingInAnnotated  ,list(df.iterrows()))
df['isAnythingInAnnotated']=resList   

#marking that we have something lacking here
df["isAnyMissing"]=df.apply(lambda row : str(row['study_id']) in  listOfDeficientStudyIds  , axis = 1) 





In [14]:


#getting sizes and spacings ...
def get_spatial_meta(row,colName):
    row=row[1]
    path=str(row[colName])
    if(len(path)>1):
        image = sitk.ReadImage(path)
        sizz= image.GetSize()
        spac= image.GetSpacing()
        orig= image.GetOrigin()
        return list(sizz)+list(spac)+list(orig)
    return []
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:    
    resList=[]
    with mp.Pool(processes = mp.cpu_count()) as pool:
        resList=pool.map(partial(get_spatial_meta,colName=keyWord)  ,list(df.iterrows()))    
    df[keyWord+'_sizz_x']= list(map(lambda arr:arr[0], resList))    
    df[keyWord+'_sizz_y']= list(map(lambda arr:arr[1], resList))    
    df[keyWord+'_sizz_z']= list(map(lambda arr:arr[2], resList))    
    df[keyWord+'_spac_x']= list(map(lambda arr:arr[3], resList))    
    df[keyWord+'_spac_y']= list(map(lambda arr:arr[4], resList))    
    df[keyWord+'_spac_z']= list(map(lambda arr:arr[5], resList))    
    df[keyWord+'_orig_x']= list(map(lambda arr:arr[6], resList))    
    df[keyWord+'_orig_y']= list(map(lambda arr:arr[7], resList))    
    df[keyWord+'_orig_z']= list(map(lambda arr:arr[8], resList))    
    

df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 
df['cor_spac_x']

In [ ]:




    

# def addPathsToDf(dff, dirDictt, keyWord):
#     return dff.apply(lambda row : findPathh(row,dirDictt ,keyWord,targetDir )   , axis = 1)

# df['t2w'] =addPathsToDf(df,dirDict, 't2w')